<a href="https://colab.research.google.com/github/midnightripper/IDEC/blob/main/idecwithmydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import scipy.io
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras import backend as K
from keras.layers import Layer, InputSpec
def dataloader(path):
    data = scipy.io.loadmat(path)
    AF = data['AF']
    modified_rows1 = AF[:-2]
    last_rows1 = AF[-2:]
    CF = data['CF']
    concatenated_array = np.concatenate((modified_rows1, CF,last_rows1), axis=0)
    new=concatenated_array
    u, count = np.unique(new[-1], return_counts=True)
    a = u[np.logical_or(count < 2, count > 2)]
    c = new[:, np.isin(new[-1], a, invert=True)]
    x = c[0:-2]; y = c[-2]; w = c[-1];
    return x.T, y.T, w.T, data['CF_info']

def normalization(feats):
    df = pd.DataFrame(feats)
    scaler = StandardScaler()
    x_new = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return x_new

In [ ]:
import numpy as np
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from scipy.optimize import linear_sum_assignment
import csv, os

def cluster_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    _, col_ind = linear_sum_assignment(w.max() - w)
    return sum([w[i, j] for i, j in zip(range(w.shape[0]), col_ind)]) * 1.0 / y_pred.size

def autoencoder(dims, act='relu'):
    n_stacks = len(dims) - 1
    x = Input(shape=(dims[0],), name='input')
    h = x
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, name='encoder_%d' % i)(h)
    h = Dense(dims[-1], name='encoder_%d' % (n_stacks - 1))(h)
    for i in range(n_stacks-1, 0, -1):
        h = Dense(dims[i], activation=act, name='decoder_%d' % i)(h)
    h = Dense(dims[0], name='decoder_0')(h)
    return Model(inputs=x, outputs=h)

In [ ]:
class ClusteringLayer(Dense):
    def __init__(self, n_clusters, alpha=1.0, **kwargs):
        self.n_clusters = n_clusters
        self.alpha = alpha
        super(ClusteringLayer, self).__init__(n_clusters, **kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=tf.float32, shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim),
                                        initializer='glorot_uniform', name='clusters')
        self.built = True

    def get_config(self):
        config = super(ClusteringLayer, self).get_config()
        config['n_clusters'] = self.n_clusters
        config['alpha'] = self.alpha
        return config

    def call(self, inputs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

In [ ]:
class DEC(object):
    def __init__(self, dims, n_clusters=2, alpha=1.0):
        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.autoencoder = autoencoder(self.dims)
        hidden = self.autoencoder.get_layer(name='encoder_%d' % (self.n_stacks - 1)).output
        self.encoder = Model(inputs=self.autoencoder.input, outputs=hidden)
        clustering_layer = ClusteringLayer(self.n_clusters, alpha=self.alpha, name='clustering')(hidden)
        self.model = Model(inputs=self.autoencoder.input, outputs=clustering_layer)

    def pretrain(self, x, batch_size=256, epochs=200, optimizer='adam'):
        self.autoencoder.compile(loss='mse', optimizer=optimizer)
        self.autoencoder.fit(x, x, batch_size=batch_size, epochs=epochs)

    def compile(self, loss='kld', optimizer='adam'):
        self.model.compile(loss=loss, optimizer=optimizer)

    def fit(self, x, y=None, batch_size=256, maxiter=2e4, tol=1e-3, update_interval=140, save_dir='./results/dec'):
        print('Update interval', update_interval)
        save_interval = int(x.shape[0] / batch_size) * 5
        print('Save interval', save_interval)
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/dec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L'])
        logwriter.writeheader()
        loss = 0
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)
                y_pred = q.argmax(1)
                if y is not None:
                    acc = cluster_acc(y, y_pred)
                    nmi = normalized_mutual_info_score(y, y_pred)
                    ari = adjusted_rand_score(y, y_pred)
                    loss = self.model.evaluate(x, p, verbose=0)
                    logwriter.writerow({'iter': ite, 'acc': acc, 'nmi': nmi, 'ari': ari, 'L': loss})
                    print('Iter-%d: ACC=%.4f, NMI=%.4f, ARI=%.4f;  L=%.5f' % (ite, acc, nmi, ari, loss))
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)
                if ite > 0 and delta_label < tol:
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break
            if (index + 1) * batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * batch_size::], y=p[index * batch_size::])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * batch_size:(index + 1) * batch_size],
                                                 y=p[index * batch_size:(index + 1) * batch_size])
                index += 1
            if ite % save_interval == 0:
                self.model.save_weights(save_dir + '/DEC_model_' + str(ite) + '.h5')
            ite += 1
        logfile.close()
        self.model.save_weights(save_dir + '/DEC_model_final.h5')
        return y_pred

    def predict_clusters(self, x):
      q = self.model.predict(x, verbose=0)
      return q.argmax(axis=1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T


In [ ]:
import argparse
original_dim = 38

# train_path = filee; test_path = filee.replace('train','test')
train_path='/content/drive/MyDrive/finalData/TypicalFA_comb1/GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
test_path='/content/drive/MyDrive/finalData/TypicalFA_comb1/GER_test_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
x, y, wtrain1, info_train1 = dataloader(train_path);
xtest, ytest, wtest1, info_test1 = dataloader(test_path);
xtest= normalization(xtest)
x= normalization(x)

# idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=args.n_clusters, batch_size=args.batch_size)
# idec.initialize_model(ae_weights=args.ae_weights, gamma=args.gamma, optimizer=optimizer)
# plot_model(idec.model, to_file='idec_model.png', show_shapes=True)
# idec.model.summary()

# # begin clustering, time not include pretraining part.
# t0 = time()
# y_pred = idec.clustering(x, y=y, tol=args.tol, maxiter=args.maxiter,
#                           update_interval=args.update_interval, save_dir=args.save_dir)
# print('acc:', cluster_acc(y, y_pred))
# print('clustering time: ', (time() - t0))

In [ ]:
# Create and Train the DEC Model
dec = DEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=2)
dec.pretrain(x, batch_size=256, epochs=200, optimizer='adam')
dec.compile(loss='kld', optimizer='adam')
dec.fit(x, y=y, batch_size=256, tol=0.001, maxiter=20000, update_interval=140, save_dir='./results/dec')

# Show the final results
y_pred = dec.predict_clusters(x)
print('ACC:', cluster_acc(y, y_pred))

Epoch 1/200
16/16 [==============================] - 7s 6ms/step - loss: 0.7500
Epoch 2/200
16/16 [==============================] - 0s 6ms/step - loss: 0.4035
Epoch 3/200
16/16 [==============================] - 0s 6ms/step - loss: 0.2605
Epoch 4/200
16/16 [==============================] - 0s 5ms/step - loss: 0.1761
Epoch 5/200
16/16 [==============================] - 0s 5ms/step - loss: 0.1337
Epoch 6/200
16/16 [==============================] - 0s 5ms/step - loss: 0.1103
Epoch 7/200
16/16 [==============================] - 0s 5ms/step - loss: 0.0941
Epoch 8/200
16/16 [==============================] - 0s 5ms/step - loss: 0.0818
Epoch 9/200
16/16 [==============================] - 0s 5ms/step - loss: 0.0742
Epoch 10/200
16/16 [==============================] - 0s 5ms/step - loss: 0.0684
Epoch 11/200
16/16 [==============================] - 0s 5ms/step - loss: 0.0655
Epoch 12/200
16/16 [==============================] - 0s 5ms/step - loss: 0.0591
Epoch 13/200
16/16 [=================

In [ ]:
from time import time
import numpy as np
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model


In [ ]:
# Assuming you have already loaded your dataset into 'x' (features) and 'y' (labels)

# Train the autoencoder
autoencoder = autoencoder([x.shape[-1], 500, 500, 2000, 10])  # Adjust dimensions as needed
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x, x, batch_size=256, epochs=50)  # Adjust batch size and epochs as needed

# Save the autoencoder weights
autoencoder.save_weights('autoencoder_weights.h5')


Epoch 1/50
16/16 [==============================] - 4s 7ms/step - loss: 0.7434
Epoch 2/50
16/16 [==============================] - 0s 6ms/step - loss: 0.3910
Epoch 3/50
16/16 [==============================] - 0s 7ms/step - loss: 0.2436
Epoch 4/50
16/16 [==============================] - 0s 6ms/step - loss: 0.1718
Epoch 5/50
16/16 [==============================] - 0s 7ms/step - loss: 0.1328
Epoch 6/50
16/16 [==============================] - 0s 7ms/step - loss: 0.1077
Epoch 7/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0932
Epoch 8/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0834
Epoch 9/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0739
Epoch 10/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0698
Epoch 11/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0637
Epoch 12/50
16/16 [==============================] - 0s 6ms/step - loss: 0.0597
Epoch 13/50
16/16 [==============================

# Try 1

In [ ]:
class IDEC(object):
    def __init__(self,
                 dims,
                 n_clusters=10,
                 alpha=1.0,
                 batch_size=256):

        super(IDEC, self).__init__()

        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.batch_size = batch_size
        self.autoencoder = autoencoder(self.dims)

    def initialize_model(self, ae_weights=None, gamma=0.1, optimizer='adam'):
        if ae_weights is not None:
            self.autoencoder.load_weights(ae_weights)
            print ('Pretrained AE weights are loaded successfully.')
        else:
            print ('ae_weights must be given. E.g.')
            print ('    python IDEC.py mnist --ae_weights weights.h5')
            exit()

        hidden = self.autoencoder.get_layer(name='encoder_%d' % (self.n_stacks - 1)).output
        self.encoder = Model(inputs=self.autoencoder.input, outputs=hidden)

        # prepare IDEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(hidden)
        self.model = Model(inputs=self.autoencoder.input,
                           outputs=[clustering_layer, self.autoencoder.output])
        self.model.compile(loss={'clustering': 'kld', 'decoder_0': 'mse'},
                           loss_weights=[gamma, 1],
                           optimizer=optimizer)

    def load_weights(self, weights_path):  # load weights of IDEC model
        self.model.load_weights(weights_path)

    def extract_feature(self, x):  # extract features from before clustering layer
        encoder = Model(self.model.input, self.model.get_layer('encoder_%d' % (self.n_stacks - 1)).output)
        return encoder.predict(x)

    def predict_clusters(self, x):  # predict cluster labels using the output of clustering layer
        q, _ = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):  # target distribution P which enhances the discrimination of soft label Q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def clustering(self, x, y=None,
                   tol=1e-3,
                   update_interval=140,
                   maxiter=2e4,
                   save_dir='./results/idec'):

        print ('Update interval'), update_interval
        save_interval = x.shape[0] / self.batch_size * 5  # 5 epochs
        print ('Save interval'), save_interval

        # initialize cluster centers using k-means
        print ('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = y_pred
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # logging file
        import csv, os
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = file(save_dir + '/idec_log.csv', 'wb')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L', 'Lc', 'Lr'])
        logwriter.writeheader()

        loss = [0, 0, 0]
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q, _ = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = y_pred
                if y is not None:
                    acc = np.round(cluster_acc(y, y_pred), 5)
                    nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
                    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
                    loss = np.round(loss, 5)
                    logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, L=loss[0], Lc=loss[1], Lr=loss[2])
                    logwriter.writerow(logdict)
                    print ('Iter'), ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; loss=', loss

                # check stop criterion
                if ite > 0 and delta_label < tol:
                    print ('delta_label '), delta_label, '< tol ', tol
                    print ('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            if (index + 1) * self.batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * self.batch_size::],
                                                 y=[p[index * self.batch_size::], x[index * self.batch_size::]])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * self.batch_size:(index + 1) * self.batch_size],
                                                 y=[p[index * self.batch_size:(index + 1) * self.batch_size],
                                                    x[index * self.batch_size:(index + 1) * self.batch_size]])
                index += 1

            # save intermediate model
            if ite % save_interval == 0:
                # save IDEC model checkpoints
                print ('saving model to:'), save_dir + '/IDEC_model_' + str(ite) + '.h5'
                self.model.save_weights(save_dir + '/IDEC_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        logfile.close()
        print ('saving model to:'), save_dir + '/IDEC_model_final.h5'
        self.model.save_weights(save_dir + '/IDEC_model_final.h5')

        return y_pred

# Try 2

In [ ]:
import numpy as np
from keras.models import Model
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.cluster import KMeans
from sklearn import metrics

class IDEC(object):
    def __init__(self,
                 dims,
                 n_clusters=10,
                 alpha=1.0,
                 batch_size=256):

        super(IDEC, self).__init__()

        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.batch_size = batch_size
        self.autoencoder = autoencoder(self.dims)

    def initialize_model(self, ae_weights=None, gamma=0.1, optimizer='adam'):
        if ae_weights is not None:
            self.autoencoder.load_weights(ae_weights)
            print('Pretrained AE weights are loaded successfully.')
        else:
            print('ae_weights must be given. E.g.')
            print('    python IDEC.py mnist --ae_weights weights.h5')
            exit()

        hidden = self.autoencoder.get_layer(name='encoder_%d' % (self.n_stacks - 1)).output
        self.encoder = Model(inputs=self.autoencoder.input, outputs=hidden)

        # prepare IDEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(hidden)
        self.model = Model(inputs=self.autoencoder.input,
                           outputs=[clustering_layer, self.autoencoder.output])
        self.model.compile(loss={'clustering': 'kld', 'decoder_0': 'mse'},
                           loss_weights=[gamma, 1],
                           optimizer=optimizer)

    def load_weights(self, weights_path):  # load weights of IDEC model
        self.model.load_weights(weights_path)

    def extract_feature(self, x):  # extract features from before clustering layer
        encoder = Model(self.model.input, self.model.get_layer('encoder_%d' % (self.n_stacks - 1)).output)
        return encoder.predict(x)

    def predict_clusters(self, x):  # predict cluster labels using the output of clustering layer
        q, _ = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):  # target distribution P which enhances the discrimination of soft label Q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def clustering(self, x, y=None,
                   tol=1e-3,
                   update_interval=140,
                   maxiter=2e4,
                   save_dir='./results/idec'):

        print('Update interval', update_interval)
        save_interval = x.shape[0] / self.batch_size * 5  # 5 epochs
        print('Save interval', save_interval)

        # initialize cluster centers using k-means
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = y_pred
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # logging file
        import csv, os
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/idec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L', 'Lc', 'Lr'])
        logwriter.writeheader()

        loss = [0, 0, 0]
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q, _ = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = y_pred
                if y is not None:
                    acc = np.round(cluster_acc(y, y_pred), 5)
                    nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
                    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
                    loss = np.round(loss, 5)
                    logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, L=loss[0], Lc=loss[1], Lr=loss[2])
                    logwriter.writerow(logdict)
                    print('Iter', ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; loss=', loss)

                # check stop criterion
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            if (index + 1) * self.batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * self.batch_size::],
                                                 y=[p[index * self.batch_size::], x[index * self.batch_size::]])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * self.batch_size:(index + 1) * self.batch_size],
                                                 y=[p[index * self.batch_size:(index + 1) * self.batch_size],
                                                    x[index * self.batch_size:(index + 1) * self.batch_size]])
                index += 1

            # save intermediate model
            if ite % save_interval == 0:
                # save IDEC model checkpoints
                print('saving model to:', save_dir + '/IDEC_model_' + str(ite) + '.h5')
                self.model.save_weights(save_dir + '/IDEC_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        logfile.close()
        print('saving model to:', save_dir + '/IDEC_model_final.h5')
        self.model.save_weights(save_dir + '/IDEC_model_final.h5')

        return y_pred


In [ ]:
x.shape

(3964, 38)

In [ ]:

class IDEC(object):
    def __init__(self, dims, n_clusters=2, alpha=1.0):
        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.autoencoder = autoencoder(self.dims)
        hidden = self.autoencoder.get_layer(name='encoder_%d' % (self.n_stacks - 1)).output
        self.encoder = Model(inputs=self.autoencoder.input, outputs=hidden)
        clustering_layer = ClusteringLayer(self.n_clusters, alpha=self.alpha, name='clustering')(hidden)
        self.model = Model(inputs=self.autoencoder.input, outputs=[clustering_layer, self.autoencoder.output])

    def pretrain(self, x, batch_size=256, epochs=200, optimizer='adam'):
        self.autoencoder.compile(loss='mse', optimizer=optimizer)
        self.autoencoder.fit(x, x, batch_size=batch_size, epochs=epochs)

    def compile(self, loss=['kld', 'mse'], loss_weights=[1, 1], optimizer='adam'):
        self.model.compile(loss=loss, loss_weights=loss_weights, optimizer=optimizer)

    def fit(self, x, y=None, batch_size=256, maxiter=2e4, tol=1e-3, update_interval=140, save_dir='./results/idec'):
        print('Update interval', update_interval)
        save_interval = int(x.shape[0] / batch_size) * 5
        print('Save interval', save_interval)
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = np.copy(y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/idec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L', 'Lc', 'Lr'])
        logwriter.writeheader()
        loss = [0, 0, 0]
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q, _ = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)
                y_pred = q.argmax(1)
                if y is not None:
                    acc = cluster_acc(y, y_pred)
                    nmi = metrics.normalized_mutual_info_score(y, y_pred)
                    ari = metrics.adjusted_rand_score(y, y_pred)
                    loss = np.round(loss, 5)
                    logdict = {'iter': ite, 'acc': acc, 'nmi': nmi, 'ari': ari, 'L': loss[0], 'Lc': loss[1], 'Lr': loss[2]}
                    logwriter.writerow(logdict)
                    print('Iter', ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; loss=', loss)
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = np.copy(y_pred)
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break
            if (index + 1) * batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * batch_size::], y=[p[index * batch_size::], x[index * batch_size::]])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * batch_size:(index + 1) * batch_size],
                                                 y=[p[index * batch_size:(index + 1) * batch_size], x[index * batch_size:(index + 1) * batch_size]])
                index += 1
            if ite % save_interval == 0:
                self.model.save_weights(save_dir + '/IDEC_model_' + str(ite) + '.h5')
            ite += 1
        logfile.close()
        self.model.save_weights(save_dir + '/IDEC_model_final.h5')
        return y_pred

    def predict_clusters(self, x):
        q, _ = self.model.predict(x, verbose=0)
        return q.argmax(axis=1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

In [ ]:
# Create and Train the IDEC Model
idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10])
idec.pretrain(x, batch_size=256, epochs=200, optimizer='adam')
idec.compile(loss=['kld', 'mse'], loss_weights=[idec.alpha, 1], optimizer='adam')
idec.fit(x, y=y, batch_size=256, tol=0.001, maxiter=20000, update_interval=140, save_dir='./results/idec')

# Show the final results
y_pred = idec.predict_clusters(x)
print('ACC:', cluster_acc(y, y_pred))


ValueError: ignored

In [ ]:
idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=2, batch_size=256)
idec.initialize_model(ae_weights='weights.h5', gamma=0.1, optimizer='adam')
plot_model(idec.model, to_file='idec_model.png', show_shapes=True)
idec.model.summary()


ValueError: ignored

In [ ]:
# Prepare the IDEC model
idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=2, batch_size=256)
idec.initialize_model(ae_weights='weights.h5', gamma=0.1, optimizer='adam')
plot_model(idec.model, to_file='idec_model.png', show_shapes=True)
idec.model.summary()

# Begin clustering, time not include pretraining part.
t0 = time()
y_pred = idec.clustering(x, y=y, tol=0.001, maxiter=2e4,
                          update_interval=140, save_dir='results/idec')
print('acc:', cluster_acc(y, y_pred))
print('clustering time: ', (time() - t0))

ValueError: ignored

In [ ]:
from time import time
import numpy as np
from keras.models import Model
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model

from sklearn.cluster import KMeans
from sklearn import metrics


class IDEC(object):
    def __init__(self,
                 dims,
                 n_clusters=10,
                 alpha=1.0,
                 batch_size=256):

        super(IDEC, self).__init__()

        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.batch_size = batch_size
        self.autoencoder = autoencoder(self.dims)

    def initialize_model(self, ae_weights=None, gamma=0.1, optimizer='adam'):
        if ae_weights is not None:
            self.autoencoder.load_weights(ae_weights)
            print('Pretrained AE weights are loaded successfully.')
        else:
            print('ae_weights must be given. E.g.')
            print('    python IDEC.py mnist --ae_weights weights.h5')
            exit()

        hidden = self.autoencoder.get_layer(name='encoder_%d' % (self.n_stacks - 1)).output
        self.encoder = Model(inputs=self.autoencoder.input, outputs=hidden)

        # prepare IDEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(hidden)
        self.model = Model(inputs=self.autoencoder.input,
                           outputs=[clustering_layer, self.autoencoder.output])
        self.model.compile(loss={'clustering': 'kld', 'decoder_0': 'mse'},
                           loss_weights=[gamma, 1],
                           optimizer=optimizer)

    def load_weights(self, weights_path):  # load weights of IDEC model
        self.model.load_weights(weights_path)

    def extract_feature(self, x):  # extract features from before clustering layer
        encoder = Model(self.model.input, self.model.get_layer('encoder_%d' % (self.n_stacks - 1)).output)
        return encoder.predict(x)

    def predict_clusters(self, x):  # predict cluster labels using the output of clustering layer
        q, _ = self.model.predict(x, verbose=0)
        return q.argmax(1)
    def dataloader(path):
      data = scipy.io.loadmat(path)
      AF = data['AF']
      modified_rows1 = AF[:-2]
      last_rows1 = AF[-2:]
      CF = data['CF']
      concatenated_array = np.concatenate((modified_rows1, CF,last_rows1), axis=0)
      new=concatenated_array
      u, count = np.unique(new[-1], return_counts=True)
      a = u[np.logical_or(count < 2, count > 2)]
      c = new[:, np.isin(new[-1], a, invert=True)]
      x = c[0:-2]; y = c[-2]; w = c[-1];
      return x.T, y.T, w.T, data['CF_info']
    @staticmethod
    def target_distribution(q):  # target distribution P which enhances the discrimination of soft label Q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def clustering(self, x, y=None,
                   tol=1e-3,
                   update_interval=140,
                   maxiter=2e4,
                   save_dir='./results/idec'):

        print('Update interval', update_interval)
        save_interval = x.shape[0] / self.batch_size * 5  # 5 epochs
        print('Save interval', save_interval)

        # initialize cluster centers using k-means
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = y_pred
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # logging file
        import csv, os
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/idec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L', 'Lc', 'Lr'])
        logwriter.writeheader()

        loss = [0, 0, 0]
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q, _ = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                y_pred = q.argmax(1)
                delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
                y_pred_last = y_pred
                if y is not None:
                    acc = np.round(cluster_acc(y, y_pred), 5)
                    nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
                    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
                    loss = np.round(loss, 5)
                    logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, L=loss[0], Lc=loss[1], Lr=loss[2])
                    logwriter.writerow(logdict)
                    print('Iter', ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; loss=', loss)

                # check stop criterion
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            if (index + 1) * self.batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * self.batch_size::],
                                                 y=[p[index * self.batch_size::], x[index * self.batch_size::]])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * self.batch_size:(index + 1) * self.batch_size],
                                                 y=[p[index * self.batch_size:(index + 1) * self.batch_size],
                                                    x[index * self.batch_size:(index + 1) * self.batch_size]])
                index += 1

            # save intermediate model
            if ite % save_interval == 0:
                # save IDEC model checkpoints
                print('saving model to:', save_dir + '/IDEC_model_' + str(ite) + '.h5')
                self.model.save_weights(save_dir + '/IDEC_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        logfile.close()
        print('saving model to:', save_dir + '/IDEC_model_final.h5')
        self.model.save_weights(save_dir + '/IDEC_model_final.h5')

        return y_pred


if __name__ == "__main__":
    # setting the hyper parameters
    import argparse

    parser = argparse.ArgumentParser(description='train',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('dataset', default='mnist', choices=['mnist', 'usps', 'reutersidf10k'])
    parser.add_argument('--n_clusters', default=10, type=int)
    parser.add_argument('--batch_size', default=256, type=int)
    parser.add_argument('--maxiter', default=2e4, type=int)
    parser.add_argument('--gamma', default=0.1, type=float,
                        help='coefficient of clustering loss')
    parser.add_argument('--update_interval', default=140, type=int)
    parser.add_argument('--tol', default=0.001, type=float)
    parser.add_argument('--ae_weights', default=None, help='This argument must be given')
    parser.add_argument('--save_dir', default='results/idec')


    # Define the dimensions for the autoencoder part of the model
    autoencoder_dims = [x.shape[-1], 500, 500, 2000]

    # Now create the IDEC model with the correct input dimensions
    idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=2, batch_size=256)
    idec.initialize_model(ae_weights='weights.h5', gamma=0.1, optimizer='adam')
    plot_model(idec.model, to_file='idec_model.png', show_shapes=True)
    idec.model.summary()

    # Begin clustering, time not include pretraining part.
    t0 = time()
    y_pred = idec.clustering(x, y=y, tol=args.tol, maxiter=args.maxiter,
                             update_interval=args.update_interval, save_dir=args.save_dir)
    print('acc:', cluster_acc(y, y_pred))
    print('clustering time: ', (time() - t0))


ValueError: ignored

In [ ]:
def calculate_accuracy(arr1, arr2):
    count = sum(1 for itr1, itr2 in zip(arr1, arr2) if itr1 == itr2)
    return count / len(arr1)

In [ ]:
print(calculate_accuracy(y,y_pred))

0.5239656912209889


In [ ]:
def flip(arr):
  result = []
  for num in arr:
      if num == 1:
          result.append(0)
      else:
          result.append(1)
  return result

In [ ]:
y_pred_flip=flip(y_pred)

In [ ]:
print(calculate_accuracy(y,y_pred_flip))

0.4760343087790111


In [ ]:
print(y_pred)

[0 1 0 ... 1 1 1]
